<a href="https://colab.research.google.com/github/CAPP30255-Project/DepressionDetection/blob/master/error_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

from google.colab import drive
ROOT = '/content/drive'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import sys
import numpy as np
from os.path import join 
repo_dir = '/content/drive/MyDrive/Repos/DepressionDetection'

In [3]:
data_dir = repo_dir + '/data/Suicide_Detection.csv'


In [4]:
%cd "{repo_dir}"
#%pip install -r jj_dt_project_requirements.txt

/content/drive/MyDrive/Repos/DepressionDetection


In [5]:
#!pip install torchtext==0.12.0

In [6]:
from data.data_process import *
from _core.bow_classifier import *
from _core.cnn_model import *

# Logistic Best Model (Glove Split)

In [6]:
depression_data = dep_data(data_dir, tokenizer = "split")
depression_data.split_data(123)

from torchtext import vocab

glove = vocab.GloVe('6B')

def bow_classifier2(data):
    counter = Counter()
    for (line, label) in data:
        counter.update(line)
    vocab = v(counter, specials = ['<unk>'], special_first = True, min_freq = 1000)
    return vocab, counter

from torchtext.vocab import vocab as v
vocab_words, counter = bow_classifier2(depression_data.all_data)
vocab_words.set_default_index(0)
glove_vector = glove.get_vecs_by_tokens(vocab_words.get_itos())


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [7]:
def collate_fn(batch):
    vectors = torch.zeros(len(batch), len(vocab_words))
    label_mapping = {'non-suicide': 0, 'suicide': 1}
    labels = []
    for index, example in enumerate(batch):
      indexes = vocab_words(example[0])
      for w_idx in indexes:
        vectors[index, int(w_idx)] = 1
      labels.append(label_mapping.get(example[1]))
    labels = torch.tensor(labels)
    return   labels.cuda(), vectors.cuda()


In [8]:
BEST_PATH_LOGISTIC = "output/logistic_glove_split_best.pt"
with open(BEST_PATH_LOGISTIC, 'rb') as f:
    logistic = torch.load(f)

In [9]:
BATCH_SIZE = 32
test_dataloader = DataLoader(depression_data.test, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

In [54]:
all_probs = []
all_labels = []
for b, (label, text) in enumerate(test_dataloader):
    log_probs = logistic(text)
    _, preds = torch.max(log_probs, 1)
    for pred in preds.tolist():
      all_probs.append(pred)
    for l in label.tolist():
      all_labels.append(l)
    if b > 4:
      break
index = list(np.array(all_probs) != np.array(all_labels))

In [57]:
wrong_preds = []
reverse_map = {'suicide': 'non-suicide', 'non-suicide': 'suicide'}
for (i, result) in enumerate(index):
  if result:
    wrong_preds.append(
        {'text' : ' '.join(
            depression_data.test[i][0]),
        'true_label' : 
         depression_data.test[i][1],
         'algorithm_label': reverse_map.get(depression_data.test[i][1])})

In [59]:
pd.DataFrame(wrong_preds).to_csv('output/logistic_glove_split_best_errors.csv', index = False)

# CNN Best Model (GloVe NLTK)

In [63]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [64]:
depression_data = dep_data(data_dir, tokenizer = "NLTK")

In [65]:
depression_data.split_data(123)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [12]:
BATCH_SIZE = 32

In [67]:
from torchtext import vocab

glove = vocab.GloVe('6B')

def bow_classifier2(data):
    counter = Counter()
    for (line, label) in data:
        counter.update(line)
    vocab = v(counter, specials = ['<unk>'], special_first = True, min_freq = 1000)
    return vocab, counter

from torchtext.vocab import vocab as v
vocab_words, counter = bow_classifier2(depression_data.all_data)
vocab_words.set_default_index(0)
glove_vector = glove.get_vecs_by_tokens(vocab_words.get_itos())

In [68]:
def collate_fn(batch):
    vectors = torch.zeros(len(batch), len(vocab_words))
    label_mapping = {'non-suicide': 0, 'suicide': 1}
    labels = []
    for index, example in enumerate(batch):
      indexes = vocab_words(example[0])
      for w_idx in indexes:
        vectors[index, int(w_idx)] = 1
      labels.append(label_mapping.get(example[1]))
    labels = torch.tensor(labels)
    return  vectors.cuda(), labels.cuda()

In [74]:
test_dataloader = DataLoader(depression_data.test, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

In [70]:
BEST_PATH_CNN = "output/CNN_glove_nltk_best.pt"
with open(BEST_PATH_CNN, 'rb') as f:
    cnn = torch.load(f)

In [77]:
all_probs = []
all_labels = []
for b, (text, label) in enumerate(test_dataloader):
    log_probs = cnn(text)
    _, preds = torch.max(log_probs, 1)
    for pred in preds.tolist():
      all_probs.append(pred)
    for l in label.tolist():
      all_labels.append(l)
    if b > 4:
      break
index = list(np.array(all_probs) != np.array(all_labels))

In [78]:
wrong_preds = []
reverse_map = {'suicide': 'non-suicide', 'non-suicide': 'suicide'}
for (i, result) in enumerate(index):
  if result:
    wrong_preds.append(
        {'text' : ' '.join(
            depression_data.test[i][0]),
        'true_label' : 
         depression_data.test[i][1],
         'algorithm_label': reverse_map.get(depression_data.test[i][1])})

In [80]:
pd.DataFrame(wrong_preds).to_csv('output/cnn_glove_nltk_best_errors.csv', index = False)

# bi-LSTM GloVe Split

In [7]:
depression_data = dep_data(data_dir, tokenizer = "split")
depression_data.split_data(123)

from torchtext import vocab

glove = vocab.GloVe('6B')

def bow_classifier2(data):
    counter = Counter()
    for (line, label) in data:
        counter.update(line)
    vocab = v(counter, specials = ['<unk>'], special_first = True, min_freq = 1000)
    return vocab, counter

from torchtext.vocab import vocab as v
vocab_words, counter = bow_classifier2(depression_data.all_data)
vocab_words.set_default_index(0)
glove_vector = glove.get_vecs_by_tokens(vocab_words.get_itos())


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


In [8]:
def collate_fn(batch):
    vectors = torch.zeros(len(batch), len(vocab_words))
    label_mapping = {'non-suicide': 0, 'suicide': 1}
    labels = []
    for index, example in enumerate(batch):
      indexes = vocab_words(example[0])
      for w_idx in indexes:
        vectors[index, int(w_idx)] = 1
      labels.append(label_mapping.get(example[1]))
    labels = torch.tensor(labels)
    return   labels.cuda(), vectors.cuda()


In [9]:
#!pip install allennlp
#!pip install --upgrade google-cloud-storage

In [10]:
BEST_PATH_LSTM = "output/biLSTM_glove_best.pt"
with open(BEST_PATH_LSTM, 'rb') as f:
    lstm = torch.load(f)

In [13]:
test_dataloader = DataLoader(depression_data.test, batch_size=BATCH_SIZE,
                             shuffle=False, 
                             collate_fn=collate_fn)

In [16]:
all_probs = []
all_labels = []
for b, (label, text) in enumerate(test_dataloader):
    log_probs = lstm(text)
    _, preds = torch.max(log_probs, 1)
    for pred in preds.tolist():
      all_probs.append(pred)
    for l in label.tolist():
      all_labels.append(l)
    if b > 4:
      break
index = list(np.array(all_probs) != np.array(all_labels))

In [17]:
wrong_preds = []
reverse_map = {'suicide': 'non-suicide', 'non-suicide': 'suicide'}
for (i, result) in enumerate(index):
  if result:
    wrong_preds.append(
        {'text' : ' '.join(
            depression_data.test[i][0]),
        'true_label' : 
         depression_data.test[i][1],
         'algorithm_label': reverse_map.get(depression_data.test[i][1])})

In [19]:
pd.DataFrame(wrong_preds).to_csv('output/LSTM_glove_split_best_errors.csv', index = False)